In [101]:
import pandas as pd
import json

In [102]:
from os import listdir
from os.path import isfile, join

mypath = '/Users/data/Documents/actas/ocr/10026/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [103]:
files.sort()
files.remove('.DS_Store')
#files

In [104]:
f = files[0]
with open(mypath+f) as json_file:
    pagina = json.load(json_file)

In [105]:
#pagina['Blocks'][0]

In [106]:
# cuáles son todos los blocktypes posibles:
blocktypes = []
for b in pagina['Blocks']:
    blocktypes.append(b['BlockType'])
    
print(set(blocktypes))

{'LINE', 'PAGE', 'WORD'}


In [107]:
# imprimo la página línea por línea
#for b in pagina['Blocks']:
#    if b['BlockType'] == 'LINE':
#        print(b['Text'])
#    else:
#        continue

In [108]:
# concateno todas las líneas para luego dividir en oraciones
full_page = []
for b in pagina['Blocks']:
    if b['BlockType'] == 'LINE':
        full_page.append(b['Text'])
    else:
        continue
        
full_page = ' '.join(full_page)
#full_page

In [109]:
#import spacy library
import spacy
  
#load core spanish library
nlp = spacy.load("es_core_news_sm")
  
#take unicode string  
#here u stands for unicode
doc = nlp(full_page)
#to print sentences
#for sent in doc.sents:
#    print(sent, '\n')

In [110]:
sentences_list = list(doc.sents)

# cuantas oraciones hay
print('Hay', len(sentences_list), 'oraciones')

# me quedo con el x%
n_keep = int(0.5*len(sentences_list))
print('Me quedo con las primeras', n_keep)

Hay 6 oraciones
Me quedo con las primeras 3


In [111]:
# cómo se vería
for i in range(n_keep):
    print(sentences_list[i], '\n')

the NOTARIA PUBLICA UNO Callegos Lic. lost Ojeda H. Cándenas, Tab. 

ADSCRIPTO VOLUMEN NUMERO 472 CUATROCIENTOS SETENTA Y DOS ESCRITURA 21578 VEINTIUN MIL QUINIENTOS SETENTA Y OCHO En la H. Ciudad de Cárdenas, Tabasco, República Mexicana; 

siendo las 12:00 doce horas del día 08 ocho del mes de marzo del año 2011 dos mil once: Yo, Licenciado JOSE ANDRES GALLEGOS TORRES, Notario Público Numero Uno del Estado y del Patrimonio Inmueble Federal, en ejercicio y con adscripción en esta ciudad, HAGO CONSTAR: Que ante mi comparecen JOSE LUIS SANDOVAL AMBROSIO, JOSE FRANCISCO PALMEROS VAZQUEZ, GABRIEL PEREZ FELIX Y TORIBIO CORTES MARTINEZ, quienes manifiestan que han decidido constituir una Sociedad Mercantil deneminada ME ISYS, SOCIEDAD ANONIMA DE CAPITAL VARIABLE y ave para efectos solicitaron y obtuvieron de la Secretaría de Relaciones Exteriores, et permiso correspondiente, mismo que en este acto doy fe de teper y que transcrito íntegramente a la letra dice: CARDENAS P ERMIS o PERMISO DE RE

In [112]:
# ese resultado lo concateno
text = ''.join(str(s) for s in sentences_list[:n_keep])
#text

In [113]:
from sentence_transformers import SentenceTransformer

#calls = sentences_list[2]

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model.max_seq_length = 512
embeddings = model.encode(str(text))

len(embeddings)

768

### Lo hago para todas las 1ras páginas

In [114]:
import glob

mypath = '/Users/data/Documents/actas/ocr/'

# esta no es la forma de buscar, sino que necesito que tengan "is_primera_pagina"
# empiezo así por ahora
first_page_files = glob.glob(mypath + "*/*000.json")

In [115]:
# le calculo el embedding a todas
import numpy as np

N = len(first_page_files)
i=0

# los guardo como columna en una matriz
# por ahora pongo a mano la longitud de los embeddings

# embedding como columna
#embeddings = np.full([768, N], np.nan)

# como fila
embeddings = np.full([N, 768], np.nan)
for f in first_page_files:
    #if i == 3:
    #    break
    print('Processing file', i+1, 'of', N)
    
    with open(f) as json_file:
        pagina = json.load(json_file)
    
    # armo la pagina
    full_page = []
    for b in pagina['Blocks']:
        if b['BlockType'] == 'LINE':
            full_page.append(b['Text'])
        else:
            continue

    full_page = ' '.join(full_page)
    
    doc = nlp(full_page)
    
    # me quedo con el primer 0.5 de las oraciones
    sentences_list = list(doc.sents)
    n_keep = int(0.5*len(sentences_list))
    text = ''.join(str(s) for s in sentences_list[:n_keep])
    
    # calculo el embedding
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    model.max_seq_length = 512
    
    # embedding como columna
    #embeddings[:,i] = model.encode(str(text))
    
    # como fila
    embeddings[i, :] = model.encode(str(text))
    
    i=i+1

Processing file 1 of 236
Processing file 2 of 236
Processing file 3 of 236
Processing file 4 of 236
Processing file 5 of 236
Processing file 6 of 236
Processing file 7 of 236
Processing file 8 of 236
Processing file 9 of 236
Processing file 10 of 236
Processing file 11 of 236
Processing file 12 of 236
Processing file 13 of 236
Processing file 14 of 236
Processing file 15 of 236
Processing file 16 of 236
Processing file 17 of 236
Processing file 18 of 236
Processing file 19 of 236
Processing file 20 of 236
Processing file 21 of 236
Processing file 22 of 236
Processing file 23 of 236
Processing file 24 of 236
Processing file 25 of 236
Processing file 26 of 236
Processing file 27 of 236
Processing file 28 of 236
Processing file 29 of 236
Processing file 30 of 236
Processing file 31 of 236
Processing file 32 of 236
Processing file 33 of 236
Processing file 34 of 236
Processing file 35 of 236
Processing file 36 of 236
Processing file 37 of 236
Processing file 38 of 236
Processing file 39 of

In [116]:
embeddings

array([[-0.16072695,  0.18971819, -0.01574366, ...,  0.06127341,
        -0.00301576,  0.07791757],
       [-0.12408128,  0.2197804 , -0.0145993 , ...,  0.03271317,
        -0.07035786,  0.00957745],
       [-0.06241748,  0.16094443, -0.01952597, ...,  0.08003706,
        -0.00485906,  0.04636014],
       ...,
       [-0.10113835,  0.16213262, -0.01417714, ...,  0.05947925,
        -0.0252417 ,  0.14009137],
       [-0.06291563,  0.18371445, -0.01929906, ...,  0.08847275,
        -0.02367591,  0.01927079],
       [-0.25925609,  0.03382551, -0.00988499, ...,  0.05020867,
        -0.11698043,  0.00723157]])

In [117]:
embeddings.shape

(236, 768)

### Exporto a tsv para visualizar en el proyector

In [118]:
import os

def save_tsv(embeddings, labels, folder_name, model_name):
    with open(os.path.join(folder_name+'/vectors_'+model_name+'.tsv'), 'w') as vectors:
        for i, emb in enumerate(embeddings):
            vectors.write("\t".join([str(num) for num in np.array(emb).tolist()]) + '\n')
    with open(os.path.join(folder_name+'/metadata_'+model_name+'.tsv'), 'w') as vectors:
        for label in labels:
            vectors.write(label + '\n')

In [119]:
save_tsv(embeddings, ['lala', 'pipi'], '/Users/data/Documents/actas/', 'model_name')

### Hacer una lista de las páginas que tengo etiquetadas

In [165]:
mypath = '/Users/data/Documents/actas/datos/'
files = [f for f in os.listdir(mypath) if isfile(os.path.join(mypath, f))]
files.remove('.DS_Store')

In [166]:
files

['project-24-at-2021-08-20-23-09-37a57061.json',
 'project-22-at-2021-08-20-23-11-8bf5dc2e.json',
 'project-23-at-2021-08-20-23-10-1f5635ab.json',
 'project-26-at-2021-08-20-23-06-62aca1f2.json',
 'project-25-at-2021-08-20-23-08-c5ee9d69.json',
 'project-20-at-2021-08-20-23-13-69739800.json']

In [167]:
# listo las primeras páginas de cada tipo
pp_constitutiva = []
pp_asamblea = []
pp_boleta = []

In [168]:
# una opción es armar el df que ya tengo, y ahí buscar las etiquetas que me interesan

# agrego estos a mano porque en el loop no salen solos
campos = ['id', 'documento', 'pagina', 'completed_by']
for f in files:
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        
        for i in range(len(ground_truth)):
            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                # acá salteo todos los que no tengan el formato que me interesa
                try:
                    campos.append(l['from_name'])
                except:
                    continue

cols = sorted(set(campos))

df_truth = pd.DataFrame(columns = cols)

for f in files:
    df_tmp = pd.DataFrame(columns = cols)
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        #print(len(ground_truth))

        for i in range(len(ground_truth)):
        #     print(i)
            identity = ground_truth[i]['id']
            documento = ground_truth[i]['file_upload'].split('-')[0]
            pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
            completed_by = ground_truth[i]['annotations'][0]['completed_by']['email']

            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                df_tmp.loc[i, 'id'] = identity
                df_tmp.loc[i, 'documento'] = documento
                df_tmp.loc[i, 'pagina'] = pagina
                df_tmp.loc[i, 'completed_by'] = completed_by

                # algunos campos aparecen en "text"
                try:
                    df_tmp.loc[i, l['from_name']] = l['value']['text'][0]
                except:
                    # otros aparecen en "choices"
                    try:
                        df_tmp.loc[i, l['from_name']] = l['value']['choices'][0]
                    except:
                        # y otros en "rectanglelabels"
                        try:
                            df_tmp.loc[i, l['from_name']] = l['value']['rectanglelabels'][0]
                        except:
                            continue
    df_truth = pd.concat([df_truth, df_tmp])
                    
# ordeno por documento y página
df_truth = df_truth.sort_values(['documento', 'pagina'])

# convierto algunos campos a int
df_truth['id'] = df_truth['id'].astype(int)
df_truth['documento'] = df_truth['documento'].astype(int)
df_truth['pagina'] = df_truth['pagina'].astype(int)

# pongo id, documento y pagina al ppio:
for c in ['pagina', 'documento', 'id']:
    first_column = df_truth.pop(c)
    df_truth.insert(0, c, first_column)

#print(df_truth.info())
df_truth.head()

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
2,8774,10227,0,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,07/08/2017,5605,acta constitutiva,Nayarit,30,Ney González Sánchez,NaN,NaN,NaN,NaN
15,8780,10227,2,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
16,8779,10227,3,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8789,10227,4,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,acta constitutiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,8783,10227,6,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion


In [169]:
# construyo el nombre del file
df_truth['file_name'] = df_truth.documento.map(str) + "-" + df_truth['pagina'].apply('{:0>3}'.format) + '.json'

In [170]:
# me quedo sólo con las páginas que me interesan
df_truth.loc[df_truth['is_primera_pagina'].notnull(), ['documento', 'pagina','is_primera_pagina', 'file_name'] ].head()

,documento,pagina,is_primera_pagina,file_name
2,10227,0,acta constitutiva,10227-000.json
10,10227,4,acta constitutiva,10227-004.json
1,10227,17,boleta de inscripcion,10227-017.json
7,10329,0,acta constitutiva,10329-000.json
3,10900,1,acta constitutiva,10900-001.json


In [204]:
# hago un dict con las 3 listas de cada cateogría

files_labeled = {}

files_labeled['acta_constitutiva'] = (df_truth.loc[df_truth['is_primera_pagina']=='acta constitutiva', 'file_name']).to_list()
files_labeled['acta_asamblea'] = (df_truth.loc[df_truth['is_primera_pagina']=='acta de asamblea', 'file_name']).to_list()
files_labeled['boleta_inscripcion'] = (df_truth.loc[df_truth['is_primera_pagina']=='boleta de inscripcion', 'file_name']).to_list()

#acta_constitutiva = (df_truth.loc[df_truth['is_primera_pagina']=='acta constitutiva', 'file_name']).to_list()
#acta_asamblea = (df_truth.loc[df_truth['is_primera_pagina']=='acta de asamblea', 'file_name']).to_list()
#boleta_inscripcion = (df_truth.loc[df_truth['is_primera_pagina']=='boleta de inscripcion', 'file_name']).to_list()

## Hay un problema con el documento 13881
En el OCR figura sólo hasta la página 30, pero tengo etiquetada la 38 como primera de boleta de inscripción

### Lo elimino a mano (revisar)

In [205]:
files_labeled['boleta_inscripcion'].remove('13881-038.json')

In [206]:
print('actas constitutivas:', len(files_labeled['acta_constitutiva']))
print('actas de asamblea:', len(files_labeled['acta_asamblea']))
print('boletas de inscripción:', len(files_labeled['boleta_inscripcion']))

actas constitutivas: 41
actas de asamblea: 3
boletas de inscripción: 27


### Calculo el embedding de estas páginas únicamente
Me tengo que hacer una lista de labels e ir agregando al array y a esa lista

### Itero sobre los modelos propuestos
Ver si está el que propuso Tofer

In [218]:
model_list = ['paraphrase-multilingual-mpnet-base-v2',
            'distiluse-base-multilingual-cased-v1',
            'paraphrase-xlm-r-multilingual-v1',
            'stsb-xlm-r-multilingual',
            'quora-distilbert-multilingual',
            'paraphrase-multilingual-MiniLM-L12-v2',
            'paraphrase-multilingual-mpnet-base-v2']

In [221]:
# más adelante loopeo sobre esto
#model_name = 'paraphrase-multilingual-mpnet-base-v2'
#model = SentenceTransformer(model_name)
#model.max_seq_length = 512

## Cómo puedo saber de antemano cuánto van a medir los embeddings?
Por ahora me armo un dict

In [224]:
embedding_length = {}

for m in model_list:
    print('Modelo', j, 'de', len(model_list), ' - ', m)
    
    model = SentenceTransformer(m)
    model.max_seq_length = 512

    # acá no me interesa loopear sobre las distintas clases
    for k in files_labeled.keys():
        print(k)

        f = files_labeled[k][0]

        full_path = glob.glob(mypath + "*/" + f)
        if len(full_path) > 1:
            raise Exception("More than one files associated to current page")

        with open(full_path[0]) as json_file:
            pagina = json.load(json_file)

        # armo la pagina
        full_page = []
        for b in pagina['Blocks']:
            if b['BlockType'] == 'LINE':
                full_page.append(b['Text'])
            else:
                continue

        full_page = ' '.join(full_page)

        doc = nlp(full_page)

        # me quedo con el primer 0.5 de las oraciones
        sentences_list = list(doc.sents)
        n_keep = int(0.5*len(sentences_list))
        text = ''.join(str(s) for s in sentences_list[:n_keep])

        # calculo el embedding
        # como fila
        embedding = model.encode(str(text))
        
        embedding_length[m] = len(embedding)

Modelo 0 de 7  -  paraphrase-multilingual-mpnet-base-v2
acta_constitutiva
acta_asamblea
boleta_inscripcion
Modelo 0 de 7  -  distiluse-base-multilingual-cased-v1
acta_constitutiva
acta_asamblea
boleta_inscripcion
Modelo 0 de 7  -  paraphrase-xlm-r-multilingual-v1


Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

acta_constitutiva
acta_asamblea
boleta_inscripcion
Modelo 0 de 7  -  paraphrase-multilingual-mpnet-base-v2
acta_constitutiva
acta_asamblea
boleta_inscripcion


In [225]:
embedding_length

{'paraphrase-multilingual-mpnet-base-v2': 768,
 'distiluse-base-multilingual-cased-v1': 512,
 'paraphrase-xlm-r-multilingual-v1': 768,
 'stsb-xlm-r-multilingual': 768,
 'quora-distilbert-multilingual': 768,
 'paraphrase-multilingual-MiniLM-L12-v2': 384}

In [226]:
mypath = '/Users/data/Documents/actas/ocr/'

N = len(acta_constitutiva) + len(acta_asamblea) + len(boleta_inscripcion)
N = N-1 # corrijo para que no me deje un vector de nans al final

# loopeo sobre los modelos
j = 0
for m in model_list:
    print('Modelo', j, 'de', len(model_list), ' - ', m)
    
    model = SentenceTransformer(m)
    model.max_seq_length = 512

    embeddings = np.full([N, embedding_length[m]], np.nan)
    labels = []

    i=0

    for k in files_labeled.keys():
        print(k)

        for f in files_labeled[k]:
            #if i == 3:
            #    break
            print('Processing file', i, 'of', N)

            full_path = glob.glob(mypath + "*/" + f)
            if len(full_path) > 1:
                raise Exception("More than one files associated to current page")

            with open(full_path[0]) as json_file:
                pagina = json.load(json_file)

            # armo la pagina
            full_page = []
            for b in pagina['Blocks']:
                if b['BlockType'] == 'LINE':
                    full_page.append(b['Text'])
                else:
                    continue

            full_page = ' '.join(full_page)

            doc = nlp(full_page)

            # me quedo con el primer 0.5 de las oraciones
            sentences_list = list(doc.sents)
            n_keep = int(0.5*len(sentences_list))
            text = ''.join(str(s) for s in sentences_list[:n_keep])

            # calculo el embedding
            # como fila
            embeddings[i, :] = model.encode(str(text))
            labels.append(k)

            i=i+1
    save_tsv(embeddings, labels, '/Users/data/Documents/actas/embedding_results/', m)

Modelo 0 de 7  -  paraphrase-multilingual-mpnet-base-v2
acta_constitutiva
Processing file 0 of 71
Processing file 1 of 71
Processing file 2 of 71
Processing file 3 of 71
Processing file 4 of 71
Processing file 5 of 71
Processing file 6 of 71
Processing file 7 of 71
Processing file 8 of 71
Processing file 9 of 71
Processing file 10 of 71
Processing file 11 of 71
Processing file 12 of 71
Processing file 13 of 71
Processing file 14 of 71
Processing file 15 of 71
Processing file 16 of 71
Processing file 17 of 71
Processing file 18 of 71
Processing file 19 of 71
Processing file 20 of 71
Processing file 21 of 71
Processing file 22 of 71
Processing file 23 of 71
Processing file 24 of 71
Processing file 25 of 71
Processing file 26 of 71
Processing file 27 of 71
Processing file 28 of 71
Processing file 29 of 71
Processing file 30 of 71
Processing file 31 of 71
Processing file 32 of 71
Processing file 33 of 71
Processing file 34 of 71
Processing file 35 of 71
Processing file 36 of 71
Processing 

Processing file 27 of 71
Processing file 28 of 71
Processing file 29 of 71
Processing file 30 of 71
Processing file 31 of 71
Processing file 32 of 71
Processing file 33 of 71
Processing file 34 of 71
Processing file 35 of 71
Processing file 36 of 71
Processing file 37 of 71
Processing file 38 of 71
Processing file 39 of 71
Processing file 40 of 71
acta_asamblea
Processing file 41 of 71
Processing file 42 of 71
Processing file 43 of 71
boleta_inscripcion
Processing file 44 of 71
Processing file 45 of 71
Processing file 46 of 71
Processing file 47 of 71
Processing file 48 of 71
Processing file 49 of 71
Processing file 50 of 71
Processing file 51 of 71
Processing file 52 of 71
Processing file 53 of 71
Processing file 54 of 71
Processing file 55 of 71
Processing file 56 of 71
Processing file 57 of 71
Processing file 58 of 71
Processing file 59 of 71
Processing file 60 of 71
Processing file 61 of 71
Processing file 62 of 71
Processing file 63 of 71
Processing file 64 of 71
Processing file 6